In [1]:
import optuna
import joblib
import optuna.visualization as vis

import os

from utils import *
from network import UNET, CustomHead
DEVICE = 'cuda'

In [2]:
def get_run(m, nsub, nsim):
    RUN = f'_m{m}_nsub{nsub}_nsim{nsim}'
    return RUN

def get_store(RUN):
    SIM_PATH = f'/nfs/scratch/eliasd/store{RUN}.zarr' 
    assert os.path.exists(f'/nfs/scratch/eliasd/store{RUN}.sync')
    store = swyft.DirectoryStore(path=SIM_PATH)
    print(f'Store has {len(store)} simulations')
    return store

def get_post(RUN, lr, factor, patience):
    save_name, save_path = get_name(RUN, lr, factor, patience)
    print(save_name)
    assert os.path.isfile(save_path)
    post = swyft.Posteriors.load(save_path).to(DEVICE)
    return post

In [3]:
RUN = get_run(0, 3, 50000)

study = joblib.load(f"studies/study{RUN}.pkl")
print("Best trial until now:")
print(" Value: ", study.best_trial.value)
print(" Params: ")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

FileNotFoundError: [Errno 2] No such file or directory: 'studies/study_m0_nsub3_nsim50000.pkl'

In [ ]:
df = study.trials_dataframe().sort_values('value')

df.to_pickle('posts_gridsearch/trials.pkl')

In [ ]:
len(df)

In [ ]:
save_names = []
posts = []
save_paths = []
for index, row in df[:10].iterrows():
    
    lr, factor, patience = row[['params_lr', 'params_factor', 'params_patience']].values
    save_name, save_path = get_name(RUN, lr, factor, patience, 'posts_gridsearch')
    
    assert os.path.isfile(save_path)
    post = swyft.Posteriors.load(save_path).to(DEVICE)
    
    save_names.append(save_name)
    posts.append(post)
    save_paths.append(save_path)

In [ ]:
for post, save_name in zip(posts, save_names):
    plot_losses(post, save_name)

In [ ]:
vis.plot_contour(study)

In [ ]:
vis.plot_param_importances(study)

In [ ]:
100e-6

In [ ]:
vis.plot_slice(study)